In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd

In [2]:
pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in c:\users\gopikrishna\anaconda3\lib\site-packages (1.0.2)
Note: you may need to restart the kernel to use updated packages.


In [3]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset = pd.get_dummies(dataset, drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
 dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [8]:
independent = dataset[[ 'Age', 'EstimatedSalary', 'Gender_Male' ]]

In [9]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [10]:
dependent = dataset[['Purchased']]

In [11]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [12]:
from sklearn.model_selection import train_test_split 
X_train, X_test, Y_train, Y_test = train_test_split(independent, dependent, test_size=1/3, random_state=0) 

In [13]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler on your data and transform it
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
from sklearn.linear_model import LogisticRegression 


In [15]:
from sklearn.model_selection import GridSearchCV 

param_grid = {
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sags'],
    'penalty' : ['none']
}
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3, n_jobs = -1,  scoring='f1_weighted')

#fitting the model for grid search
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\GOPIKRISHNA\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\GOPIKRISHNA\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\GOPIKRISHNA\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\GOPIKRISHNA\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 464, in _check_solver
   

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sags']},
             scoring='f1_weighted', verbose=3)

In [16]:
re = grid.cv_results_ 
grid_predictions = grid.predict(X_test)

from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(Y_test, grid_predictions)

from sklearn.metrics import classification_report 
clf_report = classification_report(Y_test, grid_predictions)

In [17]:
from sklearn.metrics import f1_score 
f1_macro = f1_score(Y_test, grid_predictions, average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_), f1_macro)

The f1_macro value for best parameter {'penalty': 'none', 'solver': 'newton-cg'}: 0.8872628037179481


In [18]:
print("The confusion matrix:\n", cm)

The confusion matrix:
 [[79  6]
 [ 9 40]]


In [19]:
print("The report:\n", clf_report)

The report:
               precision    recall  f1-score   support

           0       0.90      0.93      0.91        85
           1       0.87      0.82      0.84        49

    accuracy                           0.89       134
   macro avg       0.88      0.87      0.88       134
weighted avg       0.89      0.89      0.89       134



In [21]:
from sklearn.metrics import roc_auc_score 
roc_auc_score(Y_test, grid.predict_proba(X_test)[:,1])

0.9510204081632653

In [22]:
table = pd.DataFrame.from_dict(re)

In [24]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.024205,0.005801,0.001415,0.001002,none,newton-cg,"{'penalty': 'none', 'solver': 'newton-cg'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
1,0.020937,0.004717,0.001992,0.000840,none,lbfgs,"{'penalty': 'none', 'solver': 'lbfgs'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
2,0.002250,0.001650,0.000000,0.000000,none,liblinear,"{'penalty': 'none', 'solver': 'liblinear'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,0.001063,0.000966,0.000000,0.000000,none,sags,"{'penalty': 'none', 'solver': 'sags'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
